In [141]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time

def load_data(data_path, rechunk=None):
    sens_data = np.loadtxt(data_path)
    freq_arr = sens_data[:-1, 0] * 1e9
    tau_arr =  sens_data[:-1, 1]
    tb_arr = sens_data[:-1, 3]
    delta_nu = 1e6

    if rechunk is not None:
        residual = len(freq_arr) % rechunk
        data_slice = slice(-residual if (residual != 0) else None)
        freq_arr = np.mean(freq_arr[data_slice].reshape(-1, rechunk), axis=-1)
        tau_arr = np.median(tau_arr[data_slice].reshape(-1, rechunk), axis=-1)
        tb_arr = np.median(tb_arr[data_slice].reshape(-1, rechunk), axis=-1)
        delta_nu *= rechunk

    return freq_arr, tau_arr, tb_arr, delta_nu

def calc_sens(
    freq_arr,
    sky_tb_arr,
    sky_tau_arr,
    el_angle,
    inst_setup
):
    rx_grade = inst_setup["rx_grade"]
    eta_pol = inst_setup["eta_pol"]
    delta_nu = inst_setup["delta_nu"]
    t_int = inst_setup["t_int"]
    n_ants = inst_setup["n_ants"]
    ap_eff = inst_setup["ap_eff"]
    is_dsb = inst_setup["is_dsb"]

    trx_arr = (4.79924307e-11) * rx_grade * freq_arr

    trans_arr = np.exp(
        -np.minimum((sky_tau_arr / np.cos((90. - el_angle) * np.pi / 180.0)), 20.)
    )

    tb_arr = sky_tb_arr * (
        np.divide(
            1 - np.exp(sky_tau_arr * (-1 / np.cos((90 - el_angle) * np.pi / 180.))),
            1 - np.exp(-sky_tau_arr),
        )
    )

    tsys_arr = (((1 + is_dsb) * tb_arr) + (2 * trx_arr)) / trans_arr

    sens_arr = tsys_arr * (
        (97.6 / ap_eff) / np.sqrt(eta_pol * n_ants * (n_ants - 1) * delta_nu * t_int)
    )
    
    return tsys_arr, sens_arr


def print_sens(freq_arr, tsys_arr, sens_arr, freq_range_dict, title=None, delta_nu=1e6):
    if title is not None:
        print(title)
        print("")
    print("      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)")
    print("          |        [K]|      [mJy]|      [mJy]|  [Jy*km/s]")
    print("----------|-----------|-----------|-----------|-----------")
    for name, freq_lims in freq_range_dict.items():
        delta_v = (delta_nu / (1e9 * np.mean(freq_lims))) * 3e5
        freq_mask = (freq_lims[0] * 1e9 < freq_arr) & (freq_lims[1] * 1e9 > freq_arr)
        tsys_val = np.median(tsys_arr[freq_mask])
        cont_sens = 1000 * np.sum(sens_arr[freq_mask] ** -2.0) ** (-0.5)
        line_sens = 1000 * np.median(sens_arr[freq_mask]) * np.sqrt(delta_v)
        vel_sens = np.median(sens_arr[freq_mask]) * delta_v * np.sqrt(delta_v)

        print("%9s | % 9.1f | % 9.3f | % 9.1f | % 9.4f " % (name, tsys_val, cont_sens, line_sens, vel_sens))
    print(" ")
    print(" ")


In [270]:
sens_data = np.loadtxt("goodmk.out")
freq_arr = sens_data[:-1, 0] * 1e9
tx_arr =  sens_data[:-1, 2]
tb_arr = sens_data[:-1, 3]
tsys_arr, sens_arr = calc_sens(freq_arr, tb_arr, tx_arr, rx_grade=3, is_dsb=False, eta_pol=1)
print_sens(freq_arr, tsys_arr, sens_arr, "1-Hour array sensitivities (best quartile weather - 1 mm PWV)")

sens_data = np.loadtxt("avgmk.out")
freq_arr = sens_data[:-1, 0] * 1e9
tx_arr =  sens_data[:-1, 2]
tb_arr = sens_data[:-1, 3]
tsys_arr, sens_arr = calc_sens(freq_arr, tb_arr, tx_arr, rx_grade=3, is_dsb=False, eta_pol=1)
print_sens(freq_arr, tsys_arr, sens_arr, "1-Hour array sensitivities (avg weather - 2 mm PWV)")

tsys_arr, sens_arr = calc_sens(freq_arr, tb_arr, tx_arr, rx_grade=3, is_dsb=False, eta_pol=1)
print_sens(freq_arr, tsys_arr, sens_arr,"1-Hour array sensitivities (worst quartile weather - 4 mm PWV)")


1-Hour array sensitivities (best quartile weather - 1 mm PWV)

      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)
          |        [K]|      [mJy]|      [mJy]|  [Jy*km/s]
----------|-----------|-----------|-----------|-----------
   90 GHz |      40.7 |     0.079 |      22.7 |    0.0681 
  150 GHz |      58.4 |     0.081 |      26.6 |    0.0532 
  230 GHz |      93.4 |     0.118 |      34.2 |    0.0442 
  345 GHz |     217.9 |     0.368 |      65.0 |    0.0557 
  420 GHz |     715.3 |     0.672 |     195.8 |    0.1416 
  490 GHz |    1814.5 |     1.995 |     462.0 |    0.2887 
  690 GHz |    1964.3 |     2.570 |     425.8 |    0.1930 
 
 
1-Hour array sensitivities (avg weather - 2 mm PWV)

      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)
          |        [K]|      [mJy]|      [mJy]|  [Jy*km/s]
----------|-----------|-----------|-----------|-----------
   90 GHz |      43.0 |     0.083 |      24.0 |    0.0720 
  150 GHz |      66.2 |     0.092 |      30.1 |   

In [137]:
freq_arr, tau_arr, tb_arr, delta_nu = load_data("sites/SMA/avgmk.out", rechunk=10)

In [144]:
super_wsma = {
    "rx_grade": 3,
    "eta_pol": 1,
    "delta_nu": 1e7,
    "t_int": 60.,
    "n_ants": 8.,
    "ap_eff": 0.65,
    "is_dsb": True,
}

reg_sma = {
    "rx_grade": 6,
    "eta_pol": 1,
    "delta_nu": 1e7,
    "t_int": 60.,
    "n_ants": 8.,
    "ap_eff": 0.675,
    "is_dsb": True,
}

freq_range_dict = {
    "230 GHz": [224.5, 235.5],
}
#    "90 GHz": [84, 116],
#    "150 GHz": [120, 180],
#    "230 GHz": [200, 264],
#    "345 GHz": [330, 370],
#    "420 GHz": [380, 450],
#    "490 GHz": [450, 510],
#    "690 GHz": [630, 694],    
#}
utc_hrs = np.linspace(0, 24, 1440, endpoint=False)
obstime = Time(57261.9990469, format="mjd") + (utc_hrs / 24)
site = EarthLocation.of_site("Subaru")
sou_targ = SkyCoord(0, 10, unit="deg", frame="icrs", location=site, obstime=obstime)

sou_altaz = sou_targ.transform_to("altaz")
sou_altaz.alt.deg > 15.0

data_mask = freq_arr <
sub_freqs = freq_arr

for alt in sou_altaz.alt.deg:
    if alt < 15.0:
        continue
    

tsys_arr, sens_arr = calc_sens(freq_arr, tb_arr, tau_arr, 45, rx_grade=6, is_dsb=True, eta_pol=1, delta_nu=delta_nu, t_int=60.)
print_sens(freq_arr, tsys_arr, sens_arr, freq_range_dict, "1-min array sensitivities (2mm PWV; SMA Rx DSB)")

tsys_arr, sens_arr = calc_sens(freq_arr, tb_arr, tau_arr, 45, rx_grade=3, is_dsb=False, eta_pol=1, delta_nu=delta_nu, t_int=3600.)
print_sens(freq_arr, tsys_arr, sens_arr, freq_range_dict, "1-min array sensitivities (2mm PWV; wSMA Rx 2SB)")


1-min array sensitivities (2mm PWV; SMA Rx DSB)

      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)
          |        [K]|      [mJy]|      [mJy]|  [Jy*km/s]
----------|-----------|-----------|-----------|-----------
  230 GHz |     219.5 |     0.708 |      26.5 |    0.0346 
 
 
1-min array sensitivities (2mm PWV; wSMA Rx 2SB)

      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)
          |        [K]|      [mJy]|      [mJy]|  [Jy*km/s]
----------|-----------|-----------|-----------|-----------
  230 GHz |     109.7 |     0.354 |      13.3 |    0.0173 
 
 
1-min array sensitivities (2mm PWV; SMA Rx DSB)

      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)
          |        [K]|      [mJy]|      [mJy]|  [Jy*km/s]
----------|-----------|-----------|-----------|-----------
  230 GHz |     219.5 |     0.708 |      26.5 |    0.0346 
 
 
1-min array sensitivities (2mm PWV; wSMA Rx 2SB)

      Band|       Tsys|  Cont sens|  Line sens (1 km/s chan)
          |    